In [53]:

import numpy as np
import sys
import multiprocessing
import time
import os

import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import multiprocessing.pool
from datetime import datetime
import pandas as pd
import os
import numpy as np
from functools import partial
import dask.dataframe as dd
import dask.multiprocessing

LOB = pd.DataFrame()
import datetime
import os
import dask.dataframe as dd
from pathlib import Path

In [4]:
data1 = '/media/ak/Data1/'

trades_quotes_folder = os.path.join(data1, 'TradesPurchased')

processed_sampled_data = os.path.join(data1, 'ProcessedSampledData/')
tmp_folder =os.path.join(trades_quotes_folder, 'tmp')

In [15]:
tmp_files = os.listdir(tmp_folder)
symbol = 'JNI'
folder_quotes_loc = os.path.join(tmp_folder, symbol+'_quotes')
folder_trades_loc = os.path.join(tmp_folder, symbol+'_trades')
trades_dates =[ f.split(".")[0] for f in os.listdir(folder_trades_loc)]
quotes_dates =[ f.split(".")[0] for f in os.listdir(folder_quotes_loc)]

In [19]:
def intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

In [28]:
trades_and_quotes = sorted(intersection(trades_dates,quotes_dates))

In [40]:
idx =11
dateIdx = trades_and_quotes[idx]
tradesIdxLoc = os.path.join(folder_trades_loc, dateIdx+".csv")
quotesIdxLoc = os.path.join(folder_quotes_loc, dateIdx+".csv")


pd.read_csv(tradesIdxLoc)

,Unnamed: 0,Date,Time,Price,Volume
0,96423,05/20/2019,16:34:56.168645,21080.0,1.0
1,96470,05/20/2019,16:40:14.909872,21040.0,1.0
2,97536,05/20/2019,19:11:40.372648,20950.0,1.0


In [41]:
pd.read_csv(quotesIdxLoc)

,Unnamed: 0,Date,Time,Bid Price,Bid Size,Ask Price,Ask Size
0,0,05/20/2019,19:32:57.700508,18820.0,1.0,NaN,NaN
1,1,05/20/2019,21:06:38.528317,18530.0,8.0,NaN,NaN
2,2,05/20/2019,21:07:20.172252,18620.0,1.0,NaN,NaN
3,3,05/20/2019,23:26:01.776887,18630.0,8.0,NaN,NaN
4,4,05/20/2019,23:49:56.354020,18620.0,9.0,NaN,NaN


In [44]:
rawFilesLoc = '/media/ak/Data1/TradesPurchased/JNI/CSV'
rawFilesList = os.listdir(rawFilesLoc)
rawFilesIdx = 0
rawFileIdxLoc = os.path.join(rawFilesLoc, rawFilesList[rawFilesIdx])
os.path.isfile(rawFileIdxLoc)

True

In [47]:
df = pd.read_csv(rawFileIdxLoc)

In [48]:
df_quotes = df[df['Type'] =='Quote']

In [52]:
df_quotes [df_quotes['Date'] =='05/07/2019']

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
0,05/07/2019,07:09:09.933984,Quote,NaN,NaN,21750.0,6.0,22100.0,10.0
1,05/07/2019,08:00:04.081904,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
2,05/07/2019,08:29:41.675066,Quote,NaN,NaN,22100.0,2.0,22100.0,11.0
3,05/07/2019,08:30:25.802367,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
4,05/07/2019,08:31:00.122136,Quote,NaN,NaN,21970.0,1.0,21970.0,1.0
...,...,...,...,...,...,...,...,...,...
7145,05/07/2019,23:59:54.886202,Quote,NaN,NaN,21370.0,3.0,21420.0,3.0
7146,05/07/2019,23:59:56.090242,Quote,NaN,NaN,21380.0,3.0,21420.0,3.0
7147,05/07/2019,23:59:56.186612,Quote,NaN,NaN,21370.0,3.0,21420.0,3.0
7148,05/07/2019,23:59:58.390766,Quote,NaN,NaN,21380.0,3.0,21420.0,3.0


In [ ]:
def write_to_csv_for_symbol(file_loc, write_data_folder):
    ## Get symbol name:
    symbol = file_loc.split('/')[-1].split('.')[0]

    ## Need to use dask delayed if this doesn't work
    ddf = dd.read_csv(file_loc, assume_missing=True)
    quotes_ddf = ddf[ddf.Type == 'Quote']
    trades_ddf = ddf[ddf.Type == 'Trade']
    write_to_csv_for_dates(quotes_ddf, suffix='quotes', cols_to_drop=['Price', 'Volume', 'Type'],
                           output_data_folder=write_data_folder)
    write_to_csv_for_dates(trades_ddf, suffix='trades',
                           cols_to_drop=['Type', 'Bid Price', 'Bid Size', 'Ask Price', 'Ask Size'],
                           output_data_folder=write_data_folder)


def write_to_csv_for_dates(ddf, suffix, cols_to_drop, output_data_folder):
    ddf_dates = sorted(ddf['Date'].unique().compute())

    for d in ddf_dates:
        date = datetime.datetime.strptime(d, '%m/%d/%Y').date()
        quotes_for_date_ddf = ddf[ddf.Date == d].drop(cols_to_drop, axis=1)
        write_file_dir = os.path.join(output_data_folder, f"{symbol}_{suffix}")
        Path(write_file_dir).mkdir(parents=True, exist_ok=True)
        write_file_loc = os.path.join(write_file_dir, f"{datetime.datetime.strftime(date, '%Y%m%d')}.csv")
        dd.to_csv(quotes_for_date_ddf, write_file_loc, single_file=True)


In [55]:
file_loc = rawFileIdxLoc
symbol = file_loc.split('/')[-1].split('.')[0]

In [56]:
symbol

'JNIH2020'

In [114]:
ddf = dd.read_csv(file_loc, assume_missing=True)
quotes_ddf = ddf[ddf.Type == 'Quote']
trades_ddf = ddf[ddf.Type == 'Trade']

In [115]:
# ddf.head(10)

In [116]:
trades_ddf.head(10)

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
12,05/07/2019,08:45:01.268892,Trade,21830.0,1.0,NaN,NaN,NaN,NaN
75,05/07/2019,08:53:31.021930,Trade,21840.0,1.0,NaN,NaN,NaN,NaN
279,05/07/2019,09:06:01.989187,Trade,21750.0,6.0,NaN,NaN,NaN,NaN
2623,05/07/2019,13:15:03.582735,Trade,21630.0,10.0,NaN,NaN,NaN,NaN
3128,05/07/2019,14:42:11.273221,Trade,21610.0,1.0,NaN,NaN,NaN,NaN
3224,05/07/2019,14:57:13.283245,Trade,21610.0,1.0,NaN,NaN,NaN,NaN
4949,05/07/2019,22:42:16.896795,Trade,21540.0,10.0,NaN,NaN,NaN,NaN
5618,05/07/2019,23:02:04.677920,Trade,21480.0,1.0,NaN,NaN,NaN,NaN
9612,05/08/2019,02:02:30.879833,Trade,21340.0,10.0,NaN,NaN,NaN,NaN
12049,05/08/2019,04:54:00.173559,Trade,21300.0,1.0,NaN,NaN,NaN,NaN


In [82]:
quotes_ddf.head(10)
type(quotes_ddf.Date.compute().head(10)[0])

str

In [94]:
# quotes_ddf.Date.tail(10)

In [62]:
cols_to_drop=['Price', 'Volume', 'Type']

In [117]:
quotes_ddf_dates = sorted(quotes_ddf['Date'].unique().compute())

In [118]:
quotes_ddf_dates

['01/06/2020',
 '01/07/2020',
 '01/08/2020',
 '01/09/2020',
 '01/10/2020',
 '01/11/2020',
 '01/14/2020',
 '01/15/2020',
 '01/16/2020',
 '01/17/2020',
 '01/18/2020',
 '01/20/2020',
 '01/21/2020',
 '01/22/2020',
 '01/23/2020',
 '01/24/2020',
 '01/25/2020',
 '01/27/2020',
 '01/28/2020',
 '01/29/2020',
 '01/30/2020',
 '01/31/2020',
 '02/01/2020',
 '02/03/2020',
 '02/04/2020',
 '02/05/2020',
 '02/06/2020',
 '02/07/2020',
 '02/08/2020',
 '02/10/2020',
 '02/11/2020',
 '02/12/2020',
 '02/13/2020',
 '02/14/2020',
 '02/15/2020',
 '02/17/2020',
 '02/18/2020',
 '02/19/2020',
 '02/20/2020',
 '02/21/2020',
 '02/22/2020',
 '02/25/2020',
 '02/26/2020',
 '02/27/2020',
 '02/28/2020',
 '02/29/2020',
 '03/02/2020',
 '03/03/2020',
 '03/04/2020',
 '03/05/2020',
 '03/06/2020',
 '03/07/2020',
 '03/09/2020',
 '03/10/2020',
 '03/11/2020',
 '03/12/2020',
 '05/07/2019',
 '05/08/2019',
 '05/09/2019',
 '05/10/2019',
 '05/11/2019',
 '05/13/2019',
 '05/14/2019',
 '05/15/2019',
 '05/16/2019',
 '05/17/2019',
 '05/18/20

In [119]:
quotes_ddf_dates[10]

'01/18/2020'

In [120]:
quotes_ddf_dates

['01/06/2020',
 '01/07/2020',
 '01/08/2020',
 '01/09/2020',
 '01/10/2020',
 '01/11/2020',
 '01/14/2020',
 '01/15/2020',
 '01/16/2020',
 '01/17/2020',
 '01/18/2020',
 '01/20/2020',
 '01/21/2020',
 '01/22/2020',
 '01/23/2020',
 '01/24/2020',
 '01/25/2020',
 '01/27/2020',
 '01/28/2020',
 '01/29/2020',
 '01/30/2020',
 '01/31/2020',
 '02/01/2020',
 '02/03/2020',
 '02/04/2020',
 '02/05/2020',
 '02/06/2020',
 '02/07/2020',
 '02/08/2020',
 '02/10/2020',
 '02/11/2020',
 '02/12/2020',
 '02/13/2020',
 '02/14/2020',
 '02/15/2020',
 '02/17/2020',
 '02/18/2020',
 '02/19/2020',
 '02/20/2020',
 '02/21/2020',
 '02/22/2020',
 '02/25/2020',
 '02/26/2020',
 '02/27/2020',
 '02/28/2020',
 '02/29/2020',
 '03/02/2020',
 '03/03/2020',
 '03/04/2020',
 '03/05/2020',
 '03/06/2020',
 '03/07/2020',
 '03/09/2020',
 '03/10/2020',
 '03/11/2020',
 '03/12/2020',
 '05/07/2019',
 '05/08/2019',
 '05/09/2019',
 '05/10/2019',
 '05/11/2019',
 '05/13/2019',
 '05/14/2019',
 '05/15/2019',
 '05/16/2019',
 '05/17/2019',
 '05/18/20

In [130]:
for idx, _ in enumerate(quotes_ddf_dates):
    print(quotes_ddf[quotes_ddf['Date']==quotes_ddf_dates[idx]].compute().tail(10))

              Date             Time   Type  Price  Volume  Bid Price  \
503006  01/06/2020  23:59:56.455603  Quote    NaN     NaN    23170.0   
503007  01/06/2020  23:59:56.855566  Quote    NaN     NaN    23170.0   
503008  01/06/2020  23:59:57.055646  Quote    NaN     NaN    23170.0   
503009  01/06/2020  23:59:57.155640  Quote    NaN     NaN    23170.0   
503010  01/06/2020  23:59:57.255603  Quote    NaN     NaN    23170.0   
503011  01/06/2020  23:59:57.455665  Quote    NaN     NaN    23170.0   
503012  01/06/2020  23:59:57.555673  Quote    NaN     NaN    23170.0   
503013  01/06/2020  23:59:57.655688  Quote    NaN     NaN    23170.0   
503014  01/06/2020  23:59:59.455801  Quote    NaN     NaN    23170.0   
503015  01/06/2020  23:59:59.559808  Quote    NaN     NaN    23170.0   

        Bid Size  Ask Price  Ask Size  
503006     147.0    23180.0      60.0  
503007     147.0    23180.0      54.0  
503008     147.0    23180.0      61.0  
503009     147.0    23180.0      54.0  
503010 

KeyboardInterrupt: 

In [126]:
quotes_ddf.compute().Date

0          05/07/2019
1          05/07/2019
2          05/07/2019
3          05/07/2019
4          05/07/2019
              ...    
1134772    03/12/2020
1134773    03/12/2020
1134774    03/12/2020
1134776    03/12/2020
1134777    03/12/2020
Name: Date, Length: 10500683, dtype: object

In [87]:
pd.read_csv(file_loc).head(10)

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
0,05/07/2019,07:09:09.933984,Quote,NaN,NaN,21750.0,6.0,22100.0,10.0
1,05/07/2019,08:00:04.081904,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
2,05/07/2019,08:29:41.675066,Quote,NaN,NaN,22100.0,2.0,22100.0,11.0
3,05/07/2019,08:30:25.802367,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
4,05/07/2019,08:31:00.122136,Quote,NaN,NaN,21970.0,1.0,21970.0,1.0
5,05/07/2019,08:31:33.545788,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
6,05/07/2019,08:32:08.361696,Quote,NaN,NaN,21960.0,1.0,21960.0,1.0
7,05/07/2019,08:32:29.877615,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0
8,05/07/2019,08:43:49.193337,Quote,NaN,NaN,21880.0,1.0,21880.0,1.0
9,05/07/2019,08:43:51.793495,Quote,NaN,NaN,21950.0,1.0,21950.0,1.0


In [132]:
tmp_files
for _, fileIdx in enumerate(tmp_files):
    print(fileIdx)

CL_quotes
CL_trades
FLG_quotes
FLG_trades
JNI_quotes
JNI_trades
STXE_quotes
STXE_trades


In [148]:
jni_quote_files_loc = os.path.join(tmp_folder, 'JNI_quotes')
jni_trade_files_loc = os.path.join(tmp_folder, 'JNI_trades')
jni_quote_files = sorted(os.listdir(jni_quote_files_loc))
jni_trade_files = sorted(os.listdir(jni_trade_files_loc))



In [152]:
tmpIdx = 10
tmpIdxQuoteFile = os.path.join(jni_quote_files_loc, jni_quote_files[tmpIdx])
print(tmpIdxQuoteFile, os.path.isfile(tmpIdxQuoteFile))
tmpIdxTradeFile = os.path.join(jni_trade_files_loc, jni_trade_files[tmpIdx])
print(tmpIdxTradeFile, os.path.isfile(tmpIdxTradeFile))

/media/ak/Data1/TradesPurchased/tmp/JNI_quotes/20190518.csv True
/media/ak/Data1/TradesPurchased/tmp/JNI_trades/20190518.csv True


In [181]:
pd.read_csv(tmpIdxTradeFile, index_col= 0).head()

,Date,Time,Price,Volume
107649,05/18/2019,00:00:00.980518,21320.0,2.0
107650,05/18/2019,00:00:00.980518,21320.0,5.0
107651,05/18/2019,00:00:00.980518,21320.0,8.0
107652,05/18/2019,00:00:00.980518,21320.0,9.0
107653,05/18/2019,00:00:00.980518,21320.0,4.0


In [159]:
test = pd.read_csv(tmpIdxQuoteFile)

In [177]:
d = pd.to_datetime(test['Date'], infer_datetime_format=True).dt.strftime('%m/%d/%Y')

In [178]:
d

0       05/18/2019
1       05/18/2019
2       05/18/2019
3       05/18/2019
4       05/18/2019
           ...    
3305    05/18/2019
3306    05/18/2019
3307    05/18/2019
3308    05/18/2019
3309    05/18/2019
Name: Date, Length: 3310, dtype: object

In [ ]:
def load_and_format_purchased_date(symbol_, date_file_):
    symbol_quote_files_loc = os.path.join(tmp_folder, str(symbol)+'_quotes', str(date_file_)+'.csv')
    symbol_trade_files_loc = os.path.join(tmp_folder, str(symbol)+'_trades', str(date_file_)+'.csv')
    date = date_file_.split(".csv")[0]
    try:
        qt_tmp_df = pd.read_csv(symbol_quote_files_loc, index_col =0)
        trd_tmp_df = pd.read_csv(symbol_trade_files_loc, index_col =0)
        qt_tmp_df['QuoteTime'] = pd.to_datetime(qt_tmp_df.Time).values
        qt_tmp_df['Date'] = pd.to_datetime(qt_tmp_df['Date'], infer_datetime_format=True).dt.strftime('%m/%d/%Y')

    except [FileNotFoundError, AttributeError]:
        # self._logger.info(f"Data for day {date} does not exist. Skipping this day")
        pass



In [ ]:
## testing

In [268]:

def load_and_format_purchased_date(symbol_, date_file_):
    try:
        symbol_quote_files_loc = os.path.join(tmp_folder, str(symbol)+'_quotes', str(quotes_dates[10])+'.csv')
        symbol_trade_files_loc = os.path.join(tmp_folder, str(symbol)+'_trades', str(quotes_dates[10])+'.csv')
        date = date_file.split(".csv")[0]
        print(date, date_file)
        qt_tmp_df = pd.read_csv(symbol_quote_files_loc, index_col =0)
        trd_tmp_df = pd.read_csv(symbol_trade_files_loc, index_col =0)
        qt_tmp_df['QuoteTime'] = pd.to_datetime(qt_tmp_df.Time).values
        qt_tmp_df['Time'] = pd.to_datetime(qt_tmp_df.Time).values
        qt_tmp_df['Date'] = pd.to_datetime(qt_tmp_df['Date'], infer_datetime_format=True).dt.strftime('%m/%d/%Y')
        trd_tmp_df['TradeTime'] = pd.to_datetime(trd_tmp_df.Time).values
        trd_tmp_df['Time'] = pd.to_datetime(trd_tmp_df.Time).values
        trd_tmp_df['Date'] = pd.to_datetime(trd_tmp_df['Date'], infer_datetime_format=True).dt.strftime('%m/%d/%Y')
        quotes_df = qt_tmp_df.rename(columns={'Bid Price': 'BestBid', 'Ask Price': 'BestAsk',
                                              'Bid Size': 'BidSize', 'Ask Size': 'AskSize'}).fillna(method = 'bfill')
        trades_df = trd_tmp_df.rename(columns={'Price': 'TradePrice', 'Volume': 'TradeSize'}).fillna(method = 'bfill')
        trades_df = trades_df.sort_values('TradeTime')
        taq_df = pd.merge_asof( trades_df,quotes_df,on='Time',allow_exact_matches=True)

    except [FileNotFoundError, AttributeError]:
     pass
    return taq_df

In [270]:
symbol ='JNI'
#date_file = str(quotes_dates[10])
for idx in range(10,25):
    date_file = str(quotes_dates[idx])
    taq_df =load_and_format_purchased_date(symbol, date_file)
    print(taq_df.shape)


20190518 20190518
(40, 11)
20190520 20190520
(40, 11)
20190521 20190521
(40, 11)
20190522 20190522
(40, 11)
20190523 20190523
(40, 11)
20190524 20190524
(40, 11)
20190525 20190525
(40, 11)
20200128 20200128
(40, 11)
20200129 20200129
(40, 11)
20200130 20200130
(40, 11)
20200131 20200131
(40, 11)
20200201 20200201
(40, 11)
20200203 20200203
(40, 11)
20200204 20200204
(40, 11)
20200205 20200205
(40, 11)


In [271]:
# why are they all the same shape?! something is off!


In [260]:
pd.merge_asof( trades_df,quotes_df,on='Time',allow_exact_matches=True)

,Date_x,Time,TradePrice,TradeSize,TradeTime,Date_y,BestBid,BidSize,BestAsk,AskSize,QuoteTime
0,05/18/2019,2022-11-27 00:00:00.984068,21260.0,1.0,2022-11-27 00:00:00.984068,05/18/2019,21250.0,13.0,21260.0,1.0,2022-11-27 00:00:00.547887
1,05/18/2019,2022-11-27 00:04:01.626470,21270.0,5.0,2022-11-27 00:04:01.626470,05/18/2019,21260.0,11.0,21270.0,8.0,2022-11-27 00:03:59.210267
2,05/18/2019,2022-11-27 00:04:55.769900,21260.0,1.0,2022-11-27 00:04:55.769900,05/18/2019,21260.0,4.0,21270.0,11.0,2022-11-27 00:04:53.870016
3,05/18/2019,2022-11-27 00:07:14.905516,21270.0,1.0,2022-11-27 00:07:14.905516,05/18/2019,21260.0,11.0,21270.0,10.0,2022-11-27 00:07:14.217481
4,05/18/2019,2022-11-27 00:07:34.665753,21270.0,1.0,2022-11-27 00:07:34.665753,05/18/2019,21260.0,10.0,21270.0,10.0,2022-11-27 00:07:30.536732
5,05/18/2019,2022-11-27 00:07:55.537054,21270.0,1.0,2022-11-27 00:07:55.537054,05/18/2019,21260.0,11.0,21270.0,1.0,2022-11-27 00:07:54.057120
6,05/18/2019,2022-11-27 00:09:01.672413,21270.0,1.0,2022-11-27 00:09:01.672413,05/18/2019,21270.0,3.0,21280.0,11.0,2022-11-27 00:08:55.728068
7,05/18/2019,2022-11-27 00:16:32.521421,21250.0,1.0,2022-11-27 00:16:32.521421,05/18/2019,21250.0,11.0,21260.0,11.0,2022-11-27 00:16:30.981331
8,05/18/2019,2022-11-27 00:16:51.513663,21250.0,1.0,2022-11-27 00:16:51.513663,05/18/2019,21250.0,3.0,21260.0,11.0,2022-11-27 00:16:48.197167
9,05/18/2019,2022-11-27 00:22:32.658855,21250.0,1.0,2022-11-27 00:22:32.658855,05/18/2019,21240.0,11.0,21250.0,1.0,2022-11-27 00:22:31.942580


In [250]:
pd.concat([quotes_df, trades_df])

,Date,Time,BestBid,BidSize,BestAsk,AskSize,QuoteTime,TradePrice,TradeSize,TradeTime
263892,05/18/2019,2022-11-27 00:00:00.144898,21250.0,13.0,21260.0,4.0,2022-11-27 00:00:00.144898,NaN,NaN,NaT
263893,05/18/2019,2022-11-27 00:00:00.547887,21250.0,13.0,21260.0,1.0,2022-11-27 00:00:00.547887,NaN,NaN,NaT
263895,05/18/2019,2022-11-27 00:00:01.044114,21260.0,3.0,21270.0,11.0,2022-11-27 00:00:01.044114,NaN,NaN,NaT
263896,05/18/2019,2022-11-27 00:00:01.143962,21250.0,12.0,21270.0,11.0,2022-11-27 00:00:01.143962,NaN,NaN,NaT
263897,05/18/2019,2022-11-27 00:00:01.643993,21250.0,12.0,21270.0,10.0,2022-11-27 00:00:01.643993,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
273471,05/18/2019,2022-11-27 04:40:01.317226,NaN,NaN,NaN,NaN,NaT,21150.0,1.0,2022-11-27 04:40:01.317226
273601,05/18/2019,2022-11-27 04:43:03.331133,NaN,NaN,NaN,NaN,NaT,21160.0,1.0,2022-11-27 04:43:03.331133
274435,05/18/2019,2022-11-27 05:08:14.638333,NaN,NaN,NaN,NaN,NaT,21160.0,1.0,2022-11-27 05:08:14.638333
274492,05/18/2019,2022-11-27 05:16:33.717714,NaN,NaN,NaN,NaN,NaT,21160.0,1.0,2022-11-27 05:16:33.717714


In [252]:
trades_columns = ['TradeSize', 'TradePrice', 'TradeTime']
LOB = pd.merge_asof(quotes_df, trades_df.sort_values('TradeTime'),on='TradeTime',allow_exact_matches=True)

KeyError: 'TradeTime'

,Date,Time,TradePrice,TradeSize,TradeTime
94174,05/18/2019,01:23:41.496600,20780.0,2.0,2022-11-27 01:23:41.496600
94175,05/18/2019,01:23:41.500597,20790.0,2.0,2022-11-27 01:23:41.500597
94176,05/18/2019,01:23:41.500597,20800.0,1.0,2022-11-27 01:23:41.500597


In [ ]:
# def load_and_format_data(self):
#     mergedFile = dict()
#     for date in self._input_dates:
#         dateToLoad = os.path.join(date + '.csv')
#         self._logger.info(f"Loading data for:{date}")
#
#         quotesDateFile = os.path.join(self._quotes_Files, dateToLoad)
#         print(quotesDateFile)
#         tradesDateFile = os.path.join(self._trades_Files, dateToLoad)
#         print(tradesDateFile)
#
#         try:
#             qt_tmp_df = pd.read_csv(quotesDateFile, usecols=self._use_columns)
#             qt_tmp_df['TradeTime'] = pd.to_datetime(qt_tmp_df.time).values
#
#
#
#             quotes_df = quotes_df.rename(columns={'value_x': 'BestBid', 'value_y': 'BestAsk',
#                                                   'size_x': 'BidSize', 'size_y': 'AskSize', 'time_x': 'QuoteTime'})
#
#             tr_tmp_df = pd.read_csv(tradesDateFile, usecols=self._use_columns)
#             tr_tmp_df['TradeTime'] = pd.to_datetime(tr_tmp_df.time).values
#             trades_df = tr_tmp_df.dropna().fillna("ffill").sort_values('TradeTime')
#             trades_df = trades_df.rename(columns={'value': 'TradePrice',
#                                                   'size': 'TradeSize'})
#             trades_columns = ['TradeSize', 'type', 'TradePrice', 'TradeTime']
#
#             LOB = pd.merge_asof(quotes_df, trades_df[trades_columns].sort_values('TradeTime'), on='TradeTime',
#                                 allow_exact_matches=True)
#
#             LOB.BidSize = LOB['BidSize'].replace(0, 1)
#             LOB.AskSize = LOB['AskSize'].replace(0, 1)
#             LOB['TimeStamp'] = pd.to_datetime(LOB.TradeTime).dt.time
#             LOB['TradeVolume'] = LOB['TradeSize'].fillna(0)
#             LOB['total_traded_volume'] = LOB.TradeVolume
#             LOB['milliSeconds'] = [(((x.hour * 60 + x.minute) * 60 + x.second) * 1000) for x in LOB['TimeStamp']]
#             LOB['dollar_traded_volume'] = pd.Series(LOB.TradePrice * LOB.TradeVolume).fillna(0)
#             LOB['timeStampIdx'] = pd.DatetimeIndex(LOB.time_y)
#             LOB['micro_price'] = (LOB.BestAsk * LOB.AskSize + LOB.BestBid * LOB.BidSize) / (
#                     LOB.AskSize + LOB.BidSize)  #
#
#             mergedFile[date] = LOB
#
#         except [FileNotFoundError, AttributeError]:
#             self._logger.info(f"Data for day {date} does not exist. Skipping this day")
#             pass
#         return mergedFile